# [문서]초고속인터넷서비스사업자전환운영지침.hwp

## 일정
  - ~ 4/3 설계리뷰
  - ~ 5/31 단위테스트
  - 6/1 ~ 연동테스트
  - 6/30 업무전환
  - 7/3 오픈

### 업무처리흐름도

![업무처리흐름도](img/onestop-process.png)

### 업무처리 타임테이블

![업무처리흐름도](img/onestop-time-process.png)

### 업무처리 타임테이블

![민원처리흐름도](img/onestop-complain-process.png)

### 용어 정의

연관상품
- 별도의 이용약관을 통하여 가입한 서비스로 초고속인터넷서비스 해지시 사용조건(요금, 할인율 등)의 변경이 발생하고 해지와는 별개로 이용 가능한 상품
- 별도의 이용약관을 통하여 가입한 서비스로 초고속인터넷서비스 해지시 이용 불가능한 상품

부가서비스
- 초고속인터넷서비스 이용약관에 포한되어 해지시 이용 불가능한 상품

### 전환신청 양식 데이터
- 전환 신청인 정보 : 성명, 생년월일, 연락처
- 관계 : 본인, 배우자, 법정대리인
- 고객구분 : 개인, 법인
- 신청구분 : 초고속인터넷 회선 수, 유료방송상품결합 수
- 변경 전 사업자
- 변경 후 사업자
- 가입자 정보 : 성명, 생년월일(사업자등록번호), 연락번호(일반전화), 연락번호(핸드폰)
- 설치주소
- 개통희망일
- 해지희망일
- 청구정보(신용카드, 계좌번호 뒷 4자리)

### 필수 녹취사항
- 전환 신청인 정보 : 성명, 생년월일, 관계(본인여부)
- 가입자 정보 : 성명, 생년월일(사업자등록번호/법인번호), 연락처(일반전화/핸드폰)
- 변경 전 사업자, 변경 후 사업자
- 전환 신청 동의 여부
- 전환 신청 서비스(인터넷/유료방송상품 , 복수회선여부, 회선번호, 설치위치 등)
- 개통희망일, 해지희망일, 장비회수 희망일
- 청구정보(신용카드, 계좌번호 뒷 4자리)
- 미납부 이용요금, 할인반환금, 장비임대료, 기타 해지에 따른 비용
- 전환 신청에 따라 달라지는 서비스 내용(요금할인, 포인트, 등급 등)

### 민원신청 양식 데이터
- 계약자(명의인) 정보 : 성명, 생년월일, 사업자등록번호
- 신청인(대리인) 정보 : 성명, 생년월일
- 신청인 연락처 : 일반전화, 핸드폰
- 민원사항 : 불법변경/이의신청
- 변경 전 사업자, 변경 후 사업자
- 민원내용
- 민원인 정보 : 성명, 계약자(명의자)와의 관계 : 본인/배우자/법정대리인
- 민원 신청일
- 접수 정보 : 접수처, 접수일, 접수방법, 접수자, 연락처, 기타

# 개발사항정리

### 테이블 체크 사항 - 항목(컬럼)
  - 전문송수신
    - 전문일련번호
    - 수신전문의 경우 처리 상태(A:접수, F:응답완료)
    - 처리 시도 횟수
    - 등록일자(db sysdate)
  - 전환가입
    - 전환가입 일련번호(pk)
    - 전문일련번호
    - 전환가입 데이터
  - 전환해지
    - 해지일련번호
    - 담당자사번
    - 등록일자(db sysdate)
  - 전화상담
    - 일련번호(해지/민원처리 일련번호)
    - 해지/민원(별도 테이블?)
    - 통화자명
    - 통화내용
    - 통화결과(Y:성공, N:실패)
    - 상담자사번
    - 등록일자(db sysdate)
  - 민원처리
    - 민원처리일련번호
    - 수신전문의 경우 처리 상태(A:접수, F:응답완료)
    - 담당자사번
    - 등록일자(db sysdate)

### 웹화면 구성
  - 전환가입 목록
  - 전환가입 상세
  - 해지확인 전화상담(TC) 목록
  - 해지 전화상담 상세화면
  - 민원처리 목록
  - 민원처리 상세

### 연동/배치 구성
  - 소켓 전문 헤더
  - 소켓 전문 본문
  - 요청 전문 처리 여부(공통)
  - 해지가능 여부 확인 후 소켓 전송
  - 전환해지 처리 후 소켓 전송
  - 전환가입 처리 후 소켓 전송
  - 민원처리 결과 소켓 전송

# 시스템 구성 및 처리 프로세스

### GW 데몬
  - ktoa 연동 클라이언트
  - 송수신 기능, 포트 구분
  - gw 송신 api : was 에서 전달하는 전문을 ktoa 에 전송
  - gw 수신 api : ktoa 에서 전달받은 전문을 was 로 전송
  - 멀티 처리 thread

### was api
  - was 송신 api : 서비스에서 저장한 전문을 gw 송신 api 에 전달
    - 전달받은 일련번호의 서비스를 우선 처리
    - 미처리 전문을 조회하여 오래된 전문부터 처리(3분? 이상 미처리 건)
    - gw 송신 api 호출
    - 송신 전문의 처리 완료 여부를 update(일련번호의 상태 update)
    - <span style='color:red'>was 가 여러개인 경우 중복 처리 고려</span>
  - was 수신 api : gw 수신 api 에서 호출
    - 전달받은 수신 전문을 db 에 저장(미처리 상태, 일련번호 생성)
    - 미처리 전문을 조회하여 오래된 전문부터 처리(일련번호 전달)
       - 전문 번호로 해당 서비스 호출
  - 멀티처리 thread

### 서비스 처리 부분
  - 해지 가능 여무 확인, 해지 완료, 가입 처리 등
  - 송신 서비스
    - 서비스 처리 후 전문 생성 후 db 에 저장(미처리 상태, 일련번호 생성)
    - was 송신 api 호출
  - 수신 서비스
    - 전달 받은 일련번호의 처리 상태 확인
    - 미처리 서비스의 경우 처리 후 상태 update